In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pymc as pm


sns.set()

def strip(x):
    return x.replace('\t','')

global_df = pd.read_pickle('global_data.pkl')
country_dick = pd.read_pickle('country_data.pkl')
country_populations = pd.read_csv('world_pop.csv',
                                  sep=';',header=None,index_col=0,names=['population'],
                                 thousands=',',converters={0 : strip})

country_death_rate = pd.read_csv('deaths_per_year.csv',sep=';',
                                 header=None,names=['country','deaths_per_thousand'],index_col=0,usecols=[1,2])

country_death_rate.sort_index(inplace=True)
world_pop = 7.8e9
global_deaths_per_year = 56e6
global_deaths_per_day = 56e6 / 365

population_data =  country_populations.join(country_death_rate,how='outer')



In [ ]:
country_dick['Sweden'].tail()
country_death_rate.loc['Sweden'] * 10000 / 365


In [ ]:
def plot_country(country_name):
    country = country_dick[country_name]
    country_pop = population_data.loc[country_name,'population']
    country_death_rate = population_data.loc[country_name,'deaths_per_thousand'] / 1000 / 365
    country_deaths_per_day = country_death_rate * country_pop

    if country_pop > 1e6: #skip lilleput-countries
        
        mask = (1e6 * country['confirmed'] / country_pop) > 1
        country = country[mask]
        mask = country['dead_inc'] >= 1
        country = country[mask]
    
        
        country_deaths_by_corona_per_day = country['dead_inc']
        country_total_deaths_per_day =  country_deaths_per_day + country_deaths_by_corona_per_day
        ratio = country_deaths_by_corona_per_day / country_total_deaths_per_day
        #ratio = country_total_deaths_per_day / country_deaths_per_day
    
        return pd.Series(ratio,name=country_name)

death_ratio_df = pd.DataFrame()

for c in country_dick.keys():
    
    try:
        pop = population_data.loc[c] #test if present in world_population.csv
        s = plot_country(c)
        death_ratio_df = pd.concat([death_ratio_df,s],axis=1)
    except KeyError:
        print ('cant find {} in population data'.format(c))

death_ratio_df.index = pd.DatetimeIndex(death_ratio_df.index).date
mask = ~death_ratio_df.isnull().all()
df = death_ratio_df.loc[:,mask].T
last_days_status = df.iloc[:,-4:].median(axis=1)
last_days_status = last_days_status.sort_values(ascending=False)
last_days_status.dropna(inplace=True)
last_days_status *= 100


plt.figure(figsize=(18,12))
plt.title('Corona - By country ratio daily deaths caused by Corona relative to all Daily deaths [worst case]')
plt.ylabel('Corona deaths of all deaths %')
plt.axhline(last_days_status.median() , color='orange',ls='dashed',label='Median')
plt.legend (loc='upper left')


plt.savefig('corona_corona_death_ratio.jpg',format='jpg')
last_days_status.plot(kind='bar',color='crimson') #exclude San Marino
plt.savefig('corona_death_percentage.jpg',format='jpg')
last_days_status.describe()



In [ ]:
some_countries = ['China','Korea, South','Italy','Spain','US','France','Sweden']
colors = ['crimson','sienna','lime','orange','navy','b','yellow']

plt.figure(figsize=(18,12))
for i,c in enumerate(some_countries):
    country = df.loc[c] * 100
    plt.plot(country,'x--',label=c,color=colors[i])

plt.title('Corona - death ratio timeline')
plt.ylabel('ratio assumed Corona caused daily deaths relative all daily deaths in %')
plt.legend(loc='upper left')
plt.savefig('corona_death_ratio.jpg',format='jpg')

In [ ]:
plt.figure(figsize=(18,12))
for i,c in enumerate(some_countries):
    country = df.loc[c] * 100
    plt.plot(country,'x--',label=c,color=colors[i])

plt.title('Corona - death ratio timeline [log scale]')
plt.ylabel('ratio assumed Corona caused daily deaths relative all daily deaths in % [log scale]')
plt.legend(loc='upper left')
plt.yscale('log')
plt.savefig('corona_death_ratio_log.jpg',format='jpg')

In [ ]:
swe = country_dick['Sweden']
swe_excess_deaths = 100 * swe['dead_inc'] / (swe['dead_inc'] + \
                                       country_death_rate.loc['Sweden','deaths_per_thousand'] * 10000 / 365)
swe_excess_deaths

In [ ]:
swe_excess_deaths.plot(figsize=(18,12),color='crimson')
plt.axhline(swe_excess_deaths.mean(),ls='dashed',color='k',label='mean {:.2f}'.format(swe_excess_deaths.mean()))
plt.title('Sweden Excess Deaths')
plt.ylabel('%')
plt.legend(loc='upper left')

In [ ]:
#### befoe running, edit the csv-file to eliminate comments on top and bottom, and 
#### rename column dagmånad to dag_månad
#### 
scb_dead = pd.read_csv('scb-dead_per_day.csv',sep=';',encoding='ISO-8859-1')
scb_dead = scb_dead[['2020','2015-2019']]
scb_dead.index=pd.DatetimeIndex(pd.date_range('2020-01-01','2020-12-31'))

last_day = scb_dead[scb_dead['2020'] != 0]

last_day = last_day.index[-1]
last_day = last_day.date()
print (last_day)
scb_dead[:'2020-05-11']
scb_dead[:last_day]

In [ ]:

plt.figure(figsize=(18,12))
plt.title('Cumulative deaths Sweden per day - average 2015-2019 vs 2020 actuals until {} \n Data Source: SCB.se'.format(last_day))
plt.ylabel('Total Number of Deaths (cumulative)')

last_scb_actual = scb_dead['2020'].cumsum()[:last_day][-1]
print (last_scb_actual)

last_scb_mean = scb_dead['2015-2019'].cumsum()[:last_day][-1]
print (last_scb_mean)

print (last_scb_actual - last_scb_mean)

plt.plot(scb_dead['2020'].cumsum()[:last_day],ls='dashed',color='crimson',label='scb 2020 total deaths')
plt.plot(scb_dead['2015-2019'].cumsum()[:last_day],ls='dashed',color='orange',label='scb 2015-2019 mean deaths')
plt.legend(loc='upper left')
plt.savefig('scb-daily-deaths.jpg',format='jpg')

In [ ]:
scb_dead = pd.read_csv('scb-dead_per_day.csv',sep=';',encoding='ISO-8859-1')
scb_dead = scb_dead[['2020','2015-2019']]
scb_dead
scb_dead.index=pd.DatetimeIndex(pd.date_range('2020-01-01','2020-12-31'))

plt.plot(scb_dead.cumsum())

In [ ]:
#excess_deaths per day

scb_dead = scb_dead[:last_day].copy()
scb_dead['excess_deaths'] = scb_dead['2020'] - scb_dead['2015-2019']

print ('mean excess deaths',scb_dead['excess_deaths'].mean())
print ('sum excess deaths',scb_dead['excess_deaths'].sum())

print ('sum scb death data',scb_dead.sum())
excess_average_mortality = scb_dead['2020'].sum() / scb_dead['2015-2019'].sum()
print ('excess average mortality',excess_average_mortality)
print ('means scb data',scb_dead.mean())

daily_average_15_19 = scb_dead['2015-2019'].mean()
daily_std_15_19 = scb_dead['2015-2019'].std()

print ('daily average 15-19, daily std 15-19',daily_average_15_19,daily_std_15_19)
scb_dead

In [ ]:
plt.figure(figsize=(18,12))
plt.title('Excess deaths day-by-day Sweden 2020 compared to daily average 2015-2019 \n Data Source SCB.se')

plt.bar(scb_dead.index,scb_dead['excess_deaths'],
        color='orange',
        label='excess deaths per day (accumulated excess deaths YTD: {})'.format(scb_dead['excess_deaths'].sum()))

plt.axhline(scb_dead['excess_deaths'].mean(),
            color='crimson',ls='dashed',
            label='average excess deaths per day: {:.0f} daily average 2015-2019: {:.0f}'.format(
                scb_dead['excess_deaths'].mean(),daily_average_15_19))

plt.ylabel('excess deaths')
plt.legend(loc='upper left')
plt.savefig('scb_daily_excess_deaths.jpg',format='jpg')

In [ ]:
l = pm.rexponential(1 / scb_dead['2015-2019'].mean(),size=10000)
_=plt.hist(l,bins=100)
plt.axvline(l.mean(),ls='dashed',color='r')
plt.title('mean {:.2f} std {:.2f}'.format(l.mean(),l.std()))

In [ ]:
lambda_ = pm.Exponential('lambda_',1/scb_dead['2015-2019'].mean())

lkh = pm.Poisson('lkh',lambda_,observed=True,value=scb_dead['2015-2019'])

model = pm.Model([lambda_,lkh])

map_ = pm.MAP(model)
map_.fit()

mcmc = pm.MCMC(model)

sample = mcmc.sample(50000,10000,2)

lambda__post = mcmc.trace('lambda_')[:]

result = pd.DataFrame({'lambda__post' : lambda__post})



In [ ]:
print (result.describe())
result.head()

In [ ]:
nr_days = len(scb_dead)
#nr_days = 5
nr_samples_per_day = 10000

samples = np.zeros((nr_days,nr_samples_per_day))



for day in range(nr_days):
    rows = np.random.choice(result.index,replace=True,size=nr_samples_per_day)
    samples[day] = pm.rpoisson(result.iloc[rows].lambda__post)
    
samples = pd.DataFrame(samples)
idx = pd.date_range('2020-01-01',last_day)
#idx = pd.date_range('2020-01-01','2020-01-05')

samples.index = idx.date

CI_89 = np.percentile(samples,[5.5,94.5],axis=1)

#min(samples.loc[pd.Timestamp('2020-02-04').date()])
samples_mean = samples.mean(axis=1).mean()
samples_std = samples.std(axis=1).mean()

print (samples_mean,samples_std)

day1_mean = samples.iloc[0,:].mean()
day1_std = samples.iloc[0,:].std()

print (day1_mean,day1_std)
samples.iloc[0,:].plot.hist()
plt.axvline(day1_mean,ls='dashed',color='r')
plt.axvline(day1_mean - day1_std,ls='dashed',color='orange')
plt.axvline(day1_mean + day1_std,ls='dashed',color='orange')

In [ ]:
import scipy.stats as sps

foo = sps.norm.cdf(1.5) - sps.norm.cdf(-1.5)
foo

#scb_dead.drop(pd.Timestamp('2020-02-29').date(),inplace=True)

#print (scb_dead.to_string())

In [ ]:
plt.figure(figsize=(18,12))
plt.title('Expected vs Actual Deaths Y2D Sweden [ data points under 200 removed since they most likely will be updated ]')
plt.ylabel('Number of deaths')

plt.errorbar(samples.index,
             samples.mean(axis=1),
             yerr=np.array((np.abs(samples.mean(axis=1) - CI_89[0]),
                            np.abs(samples.mean(axis=1) - CI_89[1]))),fmt='o',capsize=5,label='Expected deaths, 89% CI')

mask = scb_dead['2020'] > 200 # remove impossible data points that most likely will be corrected later

plt.plot(scb_dead['2020'][mask],'o',label='actual deaths 2020',color='crimson')

plt.plot(scb_dead['2015-2019'],'o',color='orange',label='deaths 2015-2019')

plt.xticks(rotation=90)

plt.plot(scb_dead.index,np.ones_like(scb_dead['2020']) * daily_average_15_19,ls='dashed',
         color='red',label='15-19 mean')

for s in range(1,5):
    
    plt.plot(scb_dead.index,np.ones_like(scb_dead['2020']) * daily_average_15_19 + s * daily_std_15_19,
             ls='dashed',color='orange')
    plt.plot(scb_dead.index,np.ones_like(scb_dead['2020']) * daily_average_15_19 - s * daily_std_15_19,
             ls='dashed',color='orange')


plt.legend(loc='upper left')
plt.savefig('scb_expected_vs_actual_deaths.jpg',format='jpg')

In [ ]:
scb_copy = pd.read_csv('scb-dead_per_day.csv',sep=';',encoding='ISO-8859-1')
scb_copy = scb_copy[['2020','2015-2019']]
scb_copy.index=pd.DatetimeIndex(pd.date_range('2020-01-01','2020-12-31'))
scb_copy

In [ ]:

today = pd.datetime.today()
print (today)
previous_month = today - pd.DateOffset(months=1)
previous_month = previous_month.month
print (previous_month)

mask = scb_copy.index.month <= previous_month

#scb_copy = scb_copy[mask]

days_per_month = pd.DataFrame(index=range(1,13),data=[31,29,31,30,31,30,31,31,30,31,30,31],
                             columns=['days'])
days_per_month.index.name='month'

monthly_sums = scb_copy.groupby(scb_copy.index.month).sum()
monthly_sums['delta'] = monthly_sums['2020'] - monthly_sums['2015-2019']
monthly_sums['delta_pct'] = monthly_sums['2020'] / monthly_sums['2015-2019']

monthly_sums['cum_2020'] = monthly_sums['2020'].cumsum()
monthly_sums['cum_2015_2019'] = monthly_sums['2015-2019'].cumsum()
monthly_sums['cum_delta'] = monthly_sums['cum_2020'] - monthly_sums['cum_2015_2019']
monthly_sums['cum_delta_pct'] = monthly_sums['cum_2020'] / monthly_sums['cum_2015_2019']

monthly_sums[['cum_2020','cum_2015_2019']].plot.bar(figsize=(18,12),
                                                                  title='Cumulative number of deaths SWEDEN',
                                                                 color=['crimson','navy'],rot=0)

monthly_sums['2020_daily_deaths'] = monthly_sums['2020'] / days_per_month['days']
monthly_sums['base_daily_deaths'] = monthly_sums['2015-2019'] / days_per_month['days']
monthly_sums['daily_delta'] = monthly_sums['2020_daily_deaths'] - monthly_sums['base_daily_deaths']

plt.xlabel('Month')
plt.ylabel('Number of Deaths')
monthly_sums[['2020','2015-2019']].plot.bar(figsize=(18,12),title='Deaths per Month Sweden',
                                            color=['crimson','navy'],rot=0)

plt.xlabel('Month')
plt.ylabel('Number of Deaths')
monthly_sums

In [ ]:
plt.figure(figsize=(18,12))
plt.title('Total Deaths 2020 in Sweden if rest of year is like April 2020 ')
plt.plot(monthly_sums.index,monthly_sums['cum_2015_2019'],label='baseline 2015-2019')
plt.plot(monthly_sums.index,monthly_sums['cum_2020'],'x--',color='crimson',label='2020 Actual number of deaths')

plt.ylabel('Number of Deaths')
plt.xlabel('Month')

### if rest of the year is like april
###

monthly_sums.loc[5:,'2020'] = monthly_sums.loc[4,'2020']

# recalulate everything
monthly_sums['delta'] = monthly_sums['2020'] - monthly_sums['2015-2019']
monthly_sums['delta_pct'] = monthly_sums['2020'] / monthly_sums['2015-2019']

monthly_sums['cum_2020'] = monthly_sums['2020'].cumsum()
monthly_sums['cum_2015_2019'] = monthly_sums['2015-2019'].cumsum()
monthly_sums['cum_delta'] = monthly_sums['cum_2020'] - monthly_sums['cum_2015_2019']
monthly_sums['cum_delta_pct'] = monthly_sums['cum_2020'] / monthly_sums['cum_2015_2019']

plt.plot(monthly_sums.index,monthly_sums['cum_2020'],ls='dashed',
         color='orange',label='2020 if rest of year is like April')

plt.legend(loc='upper left')


monthly_sums